In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=0.1, Norm = True)
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=50
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, S_dB, df_indices, myBioSound.mps, myBioSound.wt, myBioSound.wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'y': y, 'fg_y': fg_y, 'bg_y': bg_y, 'fs': fs,
             'ps': ps, 'S_dB': S_dB, 'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'ps_fg': ps_fg, 'S_dB_fg': S_dB_fg, 'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg, 'wt_fg': wt_fg, 'wf_fg': wf_fg,
             'ps_bg': ps_bg, 'S_dB_bg': S_dB_bg, 'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg, 'wt_bg': wt_bg, 'wf_bg': wf_bg}
    
    return output
    

In [7]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')

# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),600)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')


In [8]:
# testing cell
# preproc(city_file_list[0])

In [9]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=4)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 8.442816019058228 seconds ---
--- 5.658688068389893 seconds ---
--- 6.181614875793457 seconds ---
--- 5.711115837097168 seconds ---
--- 5.754714012145996 seconds ---
--- 5.74034309387207 seconds ---
--- 5.630411148071289 seconds ---
--- 6.002909898757935 seconds ---
--- 5.794523000717163 seconds ---
--- 5.748053073883057 seconds ---
--- 5.9127678871154785 seconds ---


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


--- 8.441778898239136 seconds ---
--- 5.693047761917114 seconds ---
--- 6.137786865234375 seconds ---
--- 5.564967155456543 seconds ---
--- 5.727109909057617 seconds ---
--- 5.740365743637085 seconds ---
--- 5.633684873580933 seconds ---
--- 6.044770240783691 seconds ---
--- 5.814489126205444 seconds ---
--- 6.000507116317749 seconds ---
--- 5.947032690048218 seconds ---
--- 8.476602792739868 seconds ---
--- 5.659974098205566 seconds ---
--- 6.200077772140503 seconds ---
--- 5.934952020645142 seconds ---
--- 5.600865840911865 seconds ---
--- 5.567857980728149 seconds ---
--- 5.629659175872803 seconds ---
--- 5.895430088043213 seconds ---
--- 5.7313783168792725 seconds ---
--- 5.880213975906372 seconds ---
--- 5.850153923034668 seconds ---
--- 7.614429950714111 seconds ---
--- 7.731348991394043 seconds ---
--- 6.322110891342163 seconds ---
--- 6.1072838306427 seconds ---
--- 6.275261878967285 seconds ---
--- 6.3764731884002686 seconds ---
--- 6.517768144607544 seconds ---
--- 6.41831302

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

--- 11.898968935012817 seconds ---
--- 6.567701101303101 seconds ---
--- 6.199080228805542 seconds ---
--- 6.415242910385132 seconds ---
--- 6.169969081878662 seconds ---
--- 6.587518215179443 seconds ---
--- 6.425870180130005 seconds ---
--- 6.253186941146851 seconds ---
--- 6.248865127563477 seconds ---
--- 6.413362979888916 seconds ---
--- 6.598767042160034 seconds ---
--- 6.583969831466675 seconds ---
--- 6.366827011108398 seconds ---
--- 6.182816028594971 seconds ---
--- 6.566036939620972 seconds ---
--- 6.762310981750488 seconds ---
--- 6.455644845962524 seconds ---
--- 6.330557107925415 seconds ---
--- 6.449214935302734 seconds ---
--- 6.619786977767944 seconds ---
--- 7.049803018569946 seconds ---
--- 6.667586803436279 seconds ---
--- 6.567248106002808 seconds ---
--- 6.688320875167847 seconds ---
--- 6.570112943649292 seconds ---
--- 6.844922065734863 seconds ---
--- 6.233967065811157 seconds ---
--- 6.572413206100464 seconds ---
--- 7.064272880554199 seconds ---
--- 6.7146921

--- 12.176699161529541 seconds ---
--- 6.497125148773193 seconds ---
--- 6.139851808547974 seconds ---
--- 6.314783096313477 seconds ---
--- 6.153722763061523 seconds ---
--- 6.570952892303467 seconds ---
--- 6.3645920753479 seconds ---
--- 6.561237096786499 seconds ---
--- 6.2918360233306885 seconds ---
--- 6.303789138793945 seconds ---
--- 6.611093044281006 seconds ---
--- 6.462844133377075 seconds ---
--- 6.379651069641113 seconds ---
--- 6.3403661251068115 seconds ---
--- 6.4613940715789795 seconds ---
--- 6.733415126800537 seconds ---
--- 6.569556951522827 seconds ---
--- 6.428793668746948 seconds ---
--- 6.451390027999878 seconds ---
--- 6.649169921875 seconds ---
--- 6.94501805305481 seconds ---
--- 6.614780902862549 seconds ---
--- 6.444590091705322 seconds ---
--- 6.665884017944336 seconds ---
--- 6.551945924758911 seconds ---
--- 7.107253074645996 seconds ---
--- 6.333733797073364 seconds ---
--- 6.588622093200684 seconds ---
--- 6.889432191848755 seconds ---
--- 6.6351196765

In [10]:
len(nature_file_list)+len(city_file_list)

1980

In [6]:
for file_name in nature_file_list+city_file_list:
    print(file_name)
    run_preproc(file_name)

../data/raw/AmbisonicSoundLibrary/nature/Anna Bay Cicadas.mp3
../data/raw/AmbisonicSoundLibrary/nature/Wombat Forest Kookaburras.mp3
../data/raw/AmbisonicSoundLibrary/nature/Victoria Birds.mp3
../data/raw/AmbisonicSoundLibrary/nature/Cicadas at Dusk.mp3
../data/raw/AmbisonicSoundLibrary/nature/Torquay Surf.mp3
../data/raw/AmbisonicSoundLibrary/nature/Hurricane On Boat Deck.mp3
../data/raw/AmbisonicSoundLibrary/nature/Werribee Wetlands.mp3
../data/raw/AmbisonicSoundLibrary/nature/Daylesford Frogs.mp3
../data/raw/AmbisonicSoundLibrary/nature/Hepburn Blowhole.mp3
../data/raw/AmbisonicSoundLibrary/nature/Summer Rain Shower.mp3
../data/raw/AmbisonicSoundLibrary/nature/Guildford Cicadas.mp3
../data/raw/AmbisonicSoundLibrary/nature/Small Running Creek.mp3
../data/raw/AmbisonicSoundLibrary/nature/Waterfall Three Feet Above.mp3
../data/raw/AmbisonicSoundLibrary/nature/Strong Winds in a Forest.mp3
../data/raw/AmbisonicSoundLibrary/nature/Daylesford Spillway.mp3
../data/raw/AmbisonicSoundLibrary/

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


NoBackendError: 